In [1]:

from tensorflow.examples.tutorials.mnist import input_data

# Note that tf ver. is 1.4
import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from PIL import ImageDraw
from PIL import ImageColor
import time
from scipy.stats import norm
import pickle
import os
import cv2

%matplotlib inline
plt.style.use('ggplot')

/Users/jinpark/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/jinpark/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/jinpark/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/jinpark/anaconda3/lib/python3.6/site-packages/tensorflow/python/frame

In [2]:
def batches(batch_size, features, labels):
    output_batches = []
    sample_size = len(features)
    for start_i in range(0, sample_size, batch_size):
        end_i = start_i + batch_size
        batch = [features[start_i:end_i], labels[start_i:end_i]]
        output_batches.append(batch)
        
    return output_batches

def print_epoch_stats(epoch_i, sess, last_features, last_labels):
    """
    Print cost and validation accuracy of an epoch
    """
    current_cost = sess.run(
        cost,
        feed_dict={features: last_features, labels: last_labels})
    valid_accuracy = sess.run(
        accuracy,
        feed_dict={features: valid_features, labels: valid_labels})
    print('Epoch: {:<4} - Cost: {:<8.3} Valid Accuracy: {:<5.3}'.format(
        epoch_i,
        current_cost,
        valid_accuracy))

In [3]:
RED = 0
YELLOW = 1
GREEN = 2

# n_input = 784  # MNIST data input (img shape: 28*28)
# n_classes = 10  # MNIST total classes (0-9 digits)
n_input = 2352  # 784*3a
n_classes = 3  


directories = [
    "traffic_light_images_sim/green/",
    "traffic_light_images_sim/red/",
    "traffic_light_images_sim/yellow/"]

folders = [
    "train/",
    "valid/",
    "test/"]

dic = {"traffic_light_images_sim/green/":2,
        "traffic_light_images_sim/red/":0,
       "traffic_light_images_sim/yellow/":1}


train_features,valid_features,test_features = [],[],[]
train_labels,valid_labels,test_labels = [],[],[]


for d in directories:
    path  = os.getcwd()
    path = os.path.join(path, d)
    
    for fd in folders:
        new_path = os.path.join(path, fd)
        files = os.listdir(new_path)
        
        for f in files:
            if f.endswith("png"):
                sim_img = Image.open(new_path+f)
                rsz_image = cv2.resize(np.array(sim_img), (28,28))
                rsz_image = np.array(rsz_image).flatten()
                
                temp = np.zeros(3)
                if fd == "train/":
                    train_features.append(rsz_image)
                    temp[dic[d]]=1.
                    train_labels.append(temp.copy())
                elif fd == "valid/":
                    valid_features.append(rsz_image)
                    temp[dic[d]]=1.
                    valid_labels.append(temp.copy())
                elif fd == "test/":
                    test_features.append(rsz_image)
                    temp[dic[d]]=1.
                    test_labels.append(temp.copy())

print(len(train_features), len(valid_features), len(test_features))
print(len(train_labels), len(valid_labels), len(test_labels))
print(train_labels[0])

500 150 150
500 150 150
[0. 0. 1.]


In [4]:
# Features and Labels
features = tf.placeholder(tf.float32, [None, n_input], name="ft")
labels = tf.placeholder(tf.float32, [None, n_classes])

# Weights & bias
weights = tf.Variable(tf.random_normal([n_input, n_classes]))
bias = tf.Variable(tf.random_normal([n_classes]))

# Logits - xW + b
logits = tf.add(tf.matmul(features, weights), bias, name="op")

# Define loss and optimizer
learning_rate = tf.placeholder(tf.float32)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Calculate accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

init = tf.global_variables_initializer()

batch_size = 1
epochs = 50
learn_rate = 0.001

train_batches = batches(batch_size, train_features, train_labels)



In [5]:
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch_i in range(epochs):

        # Loop over all batches
        for batch_features, batch_labels in train_batches:
            train_feed_dict = {
                features: batch_features,
                labels: batch_labels,
                learning_rate: learn_rate}
            sess.run(optimizer, feed_dict=train_feed_dict)

        # Print cost and validation accuracy of an epoch
        print_epoch_stats(epoch_i, sess, batch_features, batch_labels)
    
    # Save my tiny model
    saver.save(sess, "./model/my_tiny_model")
    
    # Calculate accuracy for test dataset
    test_accuracy = sess.run(
        accuracy,
        feed_dict={features: test_features, labels: test_labels})

print('Test Accuracy: {}'.format(test_accuracy))

Epoch: 0    - Cost: 0.0      Valid Accuracy: 0.413
Epoch: 1    - Cost: 0.0      Valid Accuracy: 0.52 
Epoch: 2    - Cost: 0.0      Valid Accuracy: 0.5  
Epoch: 3    - Cost: 0.0      Valid Accuracy: 0.88 
Epoch: 4    - Cost: 0.0      Valid Accuracy: 0.927
Epoch: 5    - Cost: 0.0      Valid Accuracy: 0.967
Epoch: 6    - Cost: 0.0      Valid Accuracy: 0.68 
Epoch: 7    - Cost: 0.0      Valid Accuracy: 0.94 
Epoch: 8    - Cost: 0.0      Valid Accuracy: 0.94 
Epoch: 9    - Cost: 0.0      Valid Accuracy: 0.933
Epoch: 10   - Cost: 0.0      Valid Accuracy: 0.927
Epoch: 11   - Cost: 0.0      Valid Accuracy: 0.933
Epoch: 12   - Cost: 0.0      Valid Accuracy: 0.933
Epoch: 13   - Cost: 0.0      Valid Accuracy: 0.913
Epoch: 14   - Cost: 0.0      Valid Accuracy: 0.933
Epoch: 15   - Cost: 0.0      Valid Accuracy: 0.933
Epoch: 16   - Cost: 0.0      Valid Accuracy: 0.94 
Epoch: 17   - Cost: 0.0      Valid Accuracy: 0.927
Epoch: 18   - Cost: 0.0      Valid Accuracy: 0.98 
Epoch: 19   - Cost: 0.0      Va

In [7]:
with tf.Session() as sess:

    #First let's load meta graph and restore weights
    saved = tf.train.import_meta_graph('./model/my_tiny_model.meta')
    saved.restore(sess,tf.train.latest_checkpoint('./model'))


    # Now, let's access and create placeholders variables and
    # create feed-dict to feed new data

    graph = tf.get_default_graph()
    w1 = graph.get_tensor_by_name("ft:0")

    cv2_mage = Image.open("./traffic_light_images_sim/red.jpg")
    image = cv2.resize(np.array(cv2_mage), (28,28))
    image = np.array(image).flatten()     
    image = np.expand_dims(image, axis=0)

    print(image.shape)
    feed_dict ={w1:image}

    #Now, access the op that you want to run. 
    op_to_restore = graph.get_tensor_by_name("op:0")



    print(np.squeeze(sess.run(op_to_restore,feed_dict)))
    #This will print 60 which is calculated 
    #using new values of w1 and w2 and saved value of b1. 





INFO:tensorflow:Restoring parameters from ./model/my_tiny_model
(1, 2352)
[ 28122.121     -77.7169 -33981.676 ]


In [ ]:
# tf.train.write_graph(sess.graph_def,'.','model/my_tiny_graph.pb', as_text=False )
# tf.train.write_graph(sess.graph_def,'.','model/my_tiny_graph.pbtxt', as_text=True )

In [ ]:
# model_path = './model/my_tiny_graph.pb'

# graph = tf.Graph()

# config = tf.ConfigProto()
# #config.gpu_options.allow_growth = True

# with graph.as_default():
#     graph_def = tf.GraphDef()

#     with tf.gfile.GFile(model_path, 'rb') as graph_file:
#         read_graph_file = graph_file.read()

#         graph_def.ParseFromString(read_graph_file)
#         tf.import_graph_def(graph_def, name='')

#     sess = tf.Session(graph=graph, config=config)

#     restored_op = sess.graph.get_tensor_by_name('op:0')
#     restored_ft = sess.graph.get_tensor_by_name('ft:0')

In [ ]:

# cv2_image = Image.open("./ex/green.jpg")

# image = cv2.resize(np.array(cv2_image), (28,28))
# image = np.array(image).flatten()                



# with graph.as_default():
#     cv2_image_expanded = np.expand_dims(image, axis=0)

#     print(sess.run([restored_op], feed_dict={restored_ft: cv2_image_expanded}))
# #     classes = np.squeeze(classes)
# #     scores = np.squeeze(scores)

In [ ]:
# model_path = './model/my_tiny_graph.pb'
# confidence_threshold = 0.8


# graph = tf.Graph()
# config = tf.ConfigProto()
# # # config.gpu_options.allow_growth = True

# with graph.as_default():
#     graph_def = tf.GraphDef()

#     with tf.gfile.GFile(model_path, 'rb') as graph_file:
#         read_graph_file = graph_file.read()

#         graph_def.ParseFromString(read_graph_file)
#         tf.import_graph_def(graph_def, name='')

#     sess = tf.Session(graph=graph, config=config)
    
#     #sess = tf.Session(graph=graph)

#     #     image_tensor = graph.get_tensor_by_name('image_tensor:0')
#     #     num_detections = graph.get_tensor_by_name('num_detections:0')
#     #     # For each detection, it's corresponding bounding box, class and score:
#     #     # self.boxes = self.graph.get_tensor_by_name('detection_boxes:0')
#     restored_op = sess.graph.get_tensor_by_name('op:0')
#     #     scores =self.graph.get_tensor_by_name('detection_scores:0')
#     restored_ft = sess.graph.get_tensor_by_name('ft:0')



In [ ]:
# cv2_image = Image.open("./ex/green.jpg")

# image = cv2.resize(np.array(cv2_image), (28,28))
# image = np.array(image).flatten()                
# image = np.expand_dims(image, axis=0)

# with sess.graph.as_default():
#     print(sess.run(restored_op,feed_dict={restored_ft: image}))

# #    classes = np.squeeze(classes)

